# comprehension이 클 때는 generator 표현식을 고려하자

list comprehension의 문제점은 입력 시퀀스에 있는 각 값 별로 아이템을 하나씩 담은 새 리스트를 통째로 생성한다는 점이다.  
메모리를 많이 소모해서 프로그램을 망가뜨리는 원인이 된다.

예를 들어 파일을 읽고 각 줄에 있는 문자의 개수를 반환할 때,  

### list comprehension으로 하면 파일에 있는 각 줄의 길이만큼 메모리가 필요하다.

파일에 오류가 있거나 끊김이 없는 네트워크 소켓일 경우 list comprehension을 사용하면 문제가 발생한다.

다음은 입력값이 적은 경우만 처리할 수 있는 방식으로 list comprehension을 사용한 예다.

In [11]:
import random

with open('/tmp/my_file.txt', 'w') as f:
    for _ in range(10):
        f.write(random.randrange(1, 10) * 'a' + '\n')

value = [len(x) for x in open('/tmp/my_file.txt')]
print(value)

[9, 6, 7, 3, 9, 4, 10, 5, 4, 4]


### Python은 이 list comprehension과 generator를 일반화한 generator expression을 제공한다.

generator expression은 실행될 때 출력 시퀀스를 모두 구체화 한다. __대신 표현식에서 한 번에 한 아이템을 내주는 iterator로 평가된다.__  
generator expression은 () 문자 사이에 list comprehension과 유사한 문법을 사용하여 생성한다.

다음은 이전 코드와 동일한 기능을 하는 generator 표현 예다. 하지만 generator expression은 iterator로 평가되므로 더는 진행되지 않는다.

In [12]:
it = (len(x) for x in open('/tmp/my_file.txt'))
print(it)

<generator object <genexpr> at 0x7fe09191e430>


### 필요할 때 generator expression에서 다음 출력을 생성하려면 내장 함수 next로 반환받은 iterator를 한 번에 전진시키면 된다.  
코드에서 메모리 사용량을 걱정하지 않고 generator expression을 사용하면 된다.

In [13]:
print(next(it))
print(next(it))

9
6


### generator expression은 다른 generator expression과 함께 사용할 수 있다.

다음은 앞의 generator expression이 반환한 iterator를 다른 generator expression의 입력으로 사용한 예다.

In [14]:
roots = ((x, x**0.5) for x in it)

iterator를 전진시킬 때 마다 루프의 도미노 효과로 내부 iterator도 전진시키고 조건 표현식을 계산해서 입력과 출력을 처리한다.

In [15]:
print(next(roots))

(7, 2.6457513110645907)


이처럼 generator를 연결하면 Python에서 빠르게 사용할 수 있다. 큰 입력 스트림에 동작하는 기능을 결합하는 방법을 찾을 때는 최선의 도구다.  
단, generator expression이 반환한 iterator에는 상태가 있으므로 한 번 넘게 사용하지 않도록 한다.